In [80]:
import dlib
import cv2
face_detector = dlib.get_frontal_face_detector()
img = cv2.imread('./Dataset/face/hand1.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
faces = face_detector(img)
size = 80

# print("{} faces are detected.".format(len(faces)))
for f in faces:
    print("left, top, right, bottom : ", f.left(), f.top(), f.right(), f.bottom())
    # cv2.rectangle(img, (f.left(), f.top()), (f.right(), f.bottom()), (0,0,255),2)
    cv2.rectangle(img, ((f.left()-size), (f.top()-size)), ((f.right()+size),(f.bottom()+size)), (0,0,255),2)

# win = dlib.image_window()
# win.set_image(img)
# win.add_overlay(faces)

crop = img[(f.top()-size):(f.bottom()+size), (f.left()-size):(f.right()+size)]
cv2.imwrite("output.jpg", crop, [cv2.IMWRITE_JPEG_QUALITY, 80])

left, top, right, bottom :  520 281 735 496


True

In [86]:
import dlib
import cv2
import glob

face_detector = dlib.get_frontal_face_detector()
size = 60
crop_count = 0
data_path = './Dataset/Train/'

face_set = glob.glob('./Dataset/face/*.jpg')

# img = cv2.imread('./Dataset/face/hand1.jpg')
for i in range(len(face_set)):
    img = cv2.imread(face_set[i])
    faces = face_detector(img)
    
    if len(faces) > 1:
        pass
    elif len(faces) != 0:
        for f in faces:
            # print("left, top, right, bottom : ", f.left(), f.top(), f.right(), f.bottom())
            crop = img[(f.top()-size):(f.bottom()+size), (f.left()-size):(f.right()+size)]
            i = str(i)+".jpg"
            data_path_a = data_path + i
            cv2.imwrite(data_path_a, crop, [cv2.IMWRITE_JPEG_QUALITY, 80])
            crop_count += 1
            i = 0
print(len(face_set),'장의 사진 중',crop_count,'장의 얼굴 사진 Crop 완료')



9 장의 사진 중 3 장의 얼굴 사진 Crop 완료


In [76]:
face_set

['./Dataset/face\\apink1.jpg',
 './Dataset/face\\apink2.jpg',
 './Dataset/face\\golf.jpg',
 './Dataset/face\\hand1.jpg',
 './Dataset/face\\hand2.jpg',
 './Dataset/face\\kimgunwoo.jpg',
 './Dataset/face\\p1.jpg',
 './Dataset/face\\p2.jpg',
 './Dataset/face\\p3.jpg']